In [6]:
import numpy as np
import open3d as o3d
from scipy.signal import convolve2d
import redwood_3dscan as rws
import cv2

ModuleNotFoundError: No module named 'redwood_3dscan'

In [3]:
# CONSTANTS

# RGB Intrinsic Parameters
fx_rgb = 5.1885790117450188e+02
fy_rgb = 5.1946961112127485e+02
cx_rgb = 3.2558244941119034e+02
cy_rgb = 2.5373616633400465e+02

# RGB Distortion Parameters
k1_rgb =  2.0796615318809061e-01
k2_rgb = -5.8613825163911781e-01
p1_rgb = 7.2231363135888329e-04
p2_rgb = 1.0479627195765181e-03
k3_rgb = 4.9856986684705107e-01

# Depth Intrinsic Parameters
fx_d = 5.8262448167737955e+02
fy_d = 5.8269103270988637e+02
cx_d = 3.1304475870804731e+02
cy_d = 2.3844389626620386e+02

# RGB Distortion Parameters
k1_d = -9.9897236553084481e-02
k2_d = 3.9065324602765344e-01
p1_d = 1.9290592870229277e-03
p2_d = -1.9422022475975055e-03
k3_d = -5.1031725053400578e-01

### Loading sample images

### Normal Gradients

In [5]:
def get_normal_gradients(path, radius=3, max_nn=30):
    img = o3d.io.read_image(path)
    intrinsics = o3d.camera.PinholeCameraIntrinsic(
        width = img.width,
        height = img.height,
        fx = fx_rgb,
        fy = fy_rgb,
        cx = cx_rgb,
        cy = cy_rgb
    )

    # point cloud
    pc = o3d.geometry.PointCloud.create_from_rgbd_image(
        img, intrinsics
    )

    # getting surface normals
    pc.estimate_normals(
        search_params = o3d.geometry.KDTreeSearchParamHybrid(
            radius=radius, max_nn=max_nn
        )
    )

    normals = np.asarray(pc.normals)
    grad_x = convolve2d(normals[:, :, 0], [[-1, 0, 1]], mode="same", boundary="symm")
    grad_y = convolve2d(normals[:, :, 1], [[[-1], [0], [1]]], mode="same", boundary="symm")
    grad_z = convolve2d(normals[:, :, 2], [[-1, 0, 1]], mode="same", boundary="symm")

    # check this
    normal_gradients = np.stack([grad_x, grad_y, grad_z], axis=2)

    return normal_gradients

### Richer appearance

In [ ]:
def richer_appearance(path):
    image = cv2.imread(path)
    # keep a copy of the original image
    orig_image = image.copy()
    # convert to RGB image and convert to float32
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0

    edge_detector = cv2.ximgproc.createStructuredEdgeDetection('models/structured_forests.yml')
    # detect the edges
    edges = edge_detector.detectEdges(image)